In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cp "/content/drive/MyDrive/DATS 5980/commonsenseqa_validation.json" /content/

In [1]:
%cd /content/

/content


In [2]:
import os
import json
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

In [3]:
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set random seed for reproducibility across Python, NumPy, and PyTorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For deterministic behavior (slower but fully reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
dataset_path = "/content/drive/MyDrive/DATS 5980/commonsenseqa_validation.json"

with open(dataset_path, "r") as fp:
    csqa_dataset = json.load(fp)

len(csqa_dataset)

1221

In [6]:
# model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"
# model_name = "HuggingFaceTB/SmolVLM-500M-Instruct"
model_name = "HuggingFaceTB/SmolVLM-Instruct"

# model_name = "HuggingFaceTB/SmolVLM-256M-Base"

# Single Prediction

In [ ]:
# Initialize processor and model
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
def get_prediction(data_point, coco_dir, split):
    question = data_point['question']

    image_path = get_coco_path(split, data_point['image_id'], coco_dir)
    # image = load_image(image_path)
    image = Image.open(image_path).convert("RGB")

    # Create input messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": question}
            ]
        },
    ]

    # Prepare inputs
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)

    # Generate outputs
    generated_ids = model.generate(**inputs, max_new_tokens=500)
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )
    return data_point, generated_texts[0]


In [ ]:
predictions = []
for idx, data_point in enumerate(val_aokvqa_dataset):
    data_point, pred = get_prediction(data_point, coco_filtered_dir, 'val')
    print(idx, end=",")
    predictions.append((data_point, pred))
    break

0,

In [ ]:
data_point, pred

({'split': 'val',
  'image_id': 461751,
  'question_id': '22jbM6gDxdaMaunuzgrsBB',
  'question': "What is in the motorcyclist's mouth?",
  'choices': ['toothpick', 'food', 'popsicle stick', 'cigarette'],
  'correct_choice_idx': 3,
  'direct_answers': ['cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette'],
  'difficult_direct_answer': False,
  'rationales': ["He's smoking while riding.",
   'The motorcyclist has a lit cigarette in his mouth while he rides on the street.',
   'The man is smoking.']},
 "User:\n\n\n\n\nWhat is in the motorcyclist's mouth?\nAssistant: The motorcyclist is wearing a mouth guard.")

# Batch Prediction

In [7]:
# model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"

# Initialize processor and model
processor = AutoProcessor.from_pretrained(model_name)

# Force left padding for decoder-only generation
if hasattr(processor, "tokenizer"):
    processor.tokenizer.padding_side = "left"
    if processor.tokenizer.pad_token is None:
        processor.tokenizer.pad_token = processor.tokenizer.eos_token

model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [8]:
prompt = """
Question: {question}
Choices: {choices}
Give only the single correct answer. Do not include any reasoning, explanation, or extra text.
Answer (just one choice):
""".strip()

def get_batch_predictions(data_points):
    """
    Generate predictions for a batch of data points.

    Args:
        data_points (list): List of data point dicts.

    Returns:
        List of (data_point, prediction) tuples.
    """
    # Prepare images and questions
    images = []
    batch_messages = []
    for data_point in data_points:
        question = data_point['question']
        choices = data_point['choices']['text']

        mcq_question = prompt.format(question=question, choices=choices).strip()

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": mcq_question}
                ]
            }
        ]
        batch_messages.append(messages)

    # Prepare prompts for each message
    batch_prompts = [processor.apply_chat_template(msg, add_generation_prompt=True) for msg in batch_messages]

    # Prepare inputs for the batch
    batch_inputs = processor(
        text=batch_prompts,
        return_tensors="pt",
        padding=True
    )
    batch_inputs = batch_inputs.to(DEVICE)

    # Generate outputs
    batch_generated_ids = model.generate(**batch_inputs)
    batch_generated_texts = processor.batch_decode(
        batch_generated_ids,
        skip_special_tokens=True,
    )

    return list(zip(data_points, batch_generated_texts))

In [9]:
def run_batch_predictions(csqa_dataset, batch_size):
    predictions = []
    for i in range(0, len(csqa_dataset), batch_size):
        batch = csqa_dataset[i:i+batch_size]
        batch_results = get_batch_predictions(batch)
        for item in batch_results:
            predictions.append(item)
        print(f"Batch number: ({(i+1)//batch_size} / {len(csqa_dataset)/batch_size}). Progress: {(i+1) / len(csqa_dataset)*100:.2f}%")
    return predictions

In [10]:
batch_size = 6
# dataset = val_aokvqa_dataset[: 150]
dataset = csqa_dataset

predictions = run_batch_predictions(dataset, batch_size)

Batch number: (0 / 203.5). Progress: 0.08%
Batch number: (1 / 203.5). Progress: 0.57%
Batch number: (2 / 203.5). Progress: 1.06%
Batch number: (3 / 203.5). Progress: 1.56%
Batch number: (4 / 203.5). Progress: 2.05%
Batch number: (5 / 203.5). Progress: 2.54%
Batch number: (6 / 203.5). Progress: 3.03%
Batch number: (7 / 203.5). Progress: 3.52%
Batch number: (8 / 203.5). Progress: 4.01%
Batch number: (9 / 203.5). Progress: 4.50%
Batch number: (10 / 203.5). Progress: 5.00%
Batch number: (11 / 203.5). Progress: 5.49%
Batch number: (12 / 203.5). Progress: 5.98%
Batch number: (13 / 203.5). Progress: 6.47%
Batch number: (14 / 203.5). Progress: 6.96%
Batch number: (15 / 203.5). Progress: 7.45%
Batch number: (16 / 203.5). Progress: 7.94%
Batch number: (17 / 203.5). Progress: 8.44%
Batch number: (18 / 203.5). Progress: 8.93%
Batch number: (19 / 203.5). Progress: 9.42%
Batch number: (20 / 203.5). Progress: 9.91%
Batch number: (21 / 203.5). Progress: 10.40%
Batch number: (22 / 203.5). Progress: 10.

In [11]:
len(predictions)

1221

In [12]:
import pickle

model_name_mod = model_name.split("/")[1]
output_path = f"/content/drive/MyDrive/DATS 5980/val_predictions_csqa_mcq_{model_name_mod}.pkl"

with open(output_path, 'wb') as fp:
    pickle.dump(predictions, fp)

print(f"Predictions saved to {output_path}")

Predictions saved to /content/drive/MyDrive/DATS 5980/val_predictions_csqa_mcq_SmolVLM-Instruct.pkl


In [13]:
predictions[0]

({'id': '1afa02df02c908a558b4036e80242fac',
  'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
  'question_concept': 'revolving door',
  'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
   'text': ['bank', 'library', 'department store', 'mall', 'new york']},
  'answerKey': 'A'},
 "User: Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\nChoices: ['bank', 'library', 'department store', 'mall', 'new york']\nGive only the single correct answer. Do not include any reasoning, explanation, or extra text.\nAnswer (just one choice):\nAssistant: Answer: 'bank'")

In [ ]:
batch_size = 4
predictions = []
dataset = test_aokvqa_dataset[: 150]
# dataset = test_aokvqa_dataset[:2400]
# dataset = test_aokvqa_dataset[2400: 3600]
# dataset = test_aokvqa_dataset[3600: 5000]
# dataset = test_aokvqa_dataset[5000:]

print(f"dataset size: {len(dataset)}")

predictions = run_batch_predictions(dataset, batch_size, coco_filtered_dir, 'test')

dataset size: 6702
Batch number: 0. Progress: 0.67%
Batch number: 1. Progress: 3.33%
Batch number: 2. Progress: 6.00%
Batch number: 3. Progress: 8.67%
Batch number: 4. Progress: 11.33%
Batch number: 5. Progress: 14.00%
Batch number: 6. Progress: 16.67%
Batch number: 7. Progress: 19.33%
Batch number: 8. Progress: 22.00%
Batch number: 9. Progress: 24.67%
Batch number: 10. Progress: 27.33%
Batch number: 11. Progress: 30.00%
Batch number: 12. Progress: 32.67%
Batch number: 13. Progress: 35.33%
Batch number: 14. Progress: 38.00%
Batch number: 15. Progress: 40.67%
Batch number: 16. Progress: 43.33%
Batch number: 17. Progress: 46.00%
Batch number: 18. Progress: 48.67%
Batch number: 19. Progress: 51.33%
Batch number: 20. Progress: 54.00%
Batch number: 21. Progress: 56.67%
Batch number: 22. Progress: 59.33%
Batch number: 23. Progress: 62.00%
Batch number: 24. Progress: 64.67%
Batch number: 25. Progress: 67.33%
Batch number: 26. Progress: 70.00%
Batch number: 27. Progress: 72.67%
Batch number: 2

In [ ]:
len(predictions)

150

In [ ]:
import pickle

output_path = '/content/drive/MyDrive/DATS 5980/test_predictions_mcq.pkl'
with open(output_path, 'wb') as fp:
    pickle.dump(predictions, fp)

print(f"Predictions saved to {output_path}")

Predictions saved to /content/drive/MyDrive/DATS 5980/test_predictions_mcq.pkl


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Sep 17 09:51:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P0             27W /   70W |    3278MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----